In [1]:
from dotenv import load_dotenv

# .env로부터 API KEY 정보 가져오기
load_dotenv()

True

`PromptTemplate`
- 사용자 입력 및 매개변수를 언어 모델에 대한 지침으로 변환하는 데 사용되는 템플릿이다.
- LLM에 메시지를 전달하기 전에 문장 구성을 편리하게 만들어주는 기능이다.
  - `template`: 템플릿 문자열이다. 문자열 내에서 중괄호 `{}`는 변수를 나타낸다.
  - `input_variables`: 중괄호 `{}` 안에 들어갈 변수의 이름을 리스트로 정의한다.

In [3]:
from langchain_core.prompts import PromptTemplate

# template 정의
template = "{city}에 대해 알려줘."

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
# 첫 번째 방법
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['city'], template='{city}에 대해 알려줘.')

In [5]:
# 두 번째 방법
prompt_template = PromptTemplate(
  input_variables=["city"],
  template=template
)
prompt_template

PromptTemplate(input_variables=['city'], template='{city}에 대해 알려줘.')

In [ ]:
# prompt 생성
prompt = prompt_template.format(city="서울")
prompt

In [6]:
from langchain_openai import ChatOpenAI

# OpenAI Chat 전용 LLM 객체 생성
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 채팅 결과로 생성되는 토큰의 최대개수
    model_name="gpt-3.5-turbo",  # 모델명
)


## Chain 생성

### [LCEL(LangChain Expression Language)](https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel)

LCEL을 사용하여 LangChain의 기본적인 컴포넌트들을 chaining 기법을 써서 다단계 작업들의 연결 체인을 만들어 복잡한 기능을 구현할 수 있도록 해준다.

```
chain = prompt | model | output_parser
```

`|` 기호는 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달한다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달된다. 마지막으로 모델의 결과를 Parser에 전달하여 출력한다.


In [13]:
# prompt_template 객체와 llm 객체를 파이프(|) 연산자로 연결한다.
chain = prompt_template | llm

# invoke 메서드에 input을 전달한다.
# 이를 통해 AI 모델이 생성한 메시지를 반환한다.
input = {"city": "서울"}
chain.invoke(input)

AIMessage(content='서울은 대한민국의 수도이자 최대 도시로서 인구 밀도가 높고 현대적인 도시입니다. 서울은 한강을 중심으로 남북으로 나뉘어 있으며, 강남, 강북, 강동, 강서, 중구, 종로 등 다양한 지역으로 구성되어 있습니다.\n\n서울은 역사적인 유적지와 현대적인 건축물이 공존하는 도시로, 경복궁, 덕수궁, 창경궁 등의 궁궐과 남산타워, 롯데월드타워, 동대문 디자인 플라자 등의 현대적인 건축물이 많이 있습니다.\n\n또한 서울은 한국의 문화와 역사를 경험할 수 있는 다양한 박물관, 미술관, 전통시장, 쇼핑몰, 음식점, 카페 등이 있어 방문객들에게 다양한 즐길 거리를 제공합니다. 또한 대중교통 시스템이 잘 발달되어 있어 이동이 편리하고 안전한 도시로도 유명합니다.', response_metadata={'token_usage': {'completion_tokens': 376, 'prompt_tokens': 20, 'total_tokens': 396}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-69896f2e-00a4-414a-ae66-b1e6a9febde9-0')

### 출력파서(Output Parser)

`StrOutputParser`
- 출력물을 기본 str 형태로 받아볼 수 있다.

In [11]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [14]:
# 프롬프트, 모델, 출력 파서를 연결하여 체인을 구성한다.
chain = prompt_template | llm | output_parser
input = {"city": "서울"}
chain.invoke(input)

'서울은 대한민국의 수도이자 최대 도시로, 인구는 약 10백만 명으로 세계에서 가장 인구 밀도가 높은 도시 중 하나입니다. 서울은 한강을 중심으로 남북으로 이어지는 지형을 가지고 있으며, 역사적으로 중요한 장소와 현대적인 건축물이 공존하는 도시입니다.\n\n서울은 한국의 정치, 경제, 문화 중심지로서 다양한 역사적인 유적지와 문화시설이 있습니다. 광화문, 경복궁, 남산타워, 명동, 강남 등 다양한 관광 명소와 쇼핑, 먹거리, 문화 체험 등 다양한 활동을 즐길 수 있는 장소가 많이 있습니다.\n\n또한 서울은 대한민국의 교통 중심지로서 국내외로의 교통이 편리하고 다양한 교통수단이 잘 발달해 있습니다. 또한 서울은 한국의 주요 대학이 위치해 있어 교육도 중요한 역할을 하고 있습니다.\n\n서울은 한국의 역사와 현대 문화가 공존하는 도시로, 다양한 경험과 즐길 거리가 많은 도시입니다.'